In [78]:
import pandas as pd
import numpy as np

# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) # Confirm folder created
os.listdir("Data/")



['.DS_Store',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'title-akas-us-only.csv',
 'title_akas.csv.gz']

In [79]:
# Load the three files into respective dataframes
url_akas = "https://datasets.imdbws.com/title.akas.tsv.gz"
url_basics = "https://datasets.imdbws.com/title.basics.tsv.gz"
url_ratings = "https://datasets.imdbws.com/title.ratings.tsv.gz"

akas_df = pd.read_csv(url_akas, compression='gzip', sep='\t', low_memory=False)
basics_df = pd.read_csv(url_basics, compression='gzip', sep='\t', low_memory=False)
ratings_df = pd.read_csv(url_ratings, compression='gzip', sep='\t', low_memory=False)

# Display the first few rows of each dataframe to understand their structure
akas_df.head(), basics_df.head(), ratings_df.head()


(     titleId  ordering                      title region language  \
 0  tt0000001         1                 Карменсіта     UA       \N   
 1  tt0000001         2                 Carmencita     DE       \N   
 2  tt0000001         3  Carmencita - spanyol tánc     HU       \N   
 3  tt0000001         4                 Καρμενσίτα     GR       \N   
 4  tt0000001         5                 Карменсита     RU       \N   
 
          types     attributes isOriginalTitle  
 0  imdbDisplay             \N               0  
 1           \N  literal title               0  
 2  imdbDisplay             \N               0  
 3  imdbDisplay             \N               0  
 4  imdbDisplay             \N               0  ,
       tconst titleType            primaryTitle           originalTitle  \
 0  tt0000001     short              Carmencita              Carmencita   
 1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
 2  tt0000003     short          Pauvre Pierrot          P

In [80]:
akas_df = akas_df[(akas_df['region'] == 'US')]

In [81]:
akas_df.replace({'\\N': np.nan}, inplace=True)

# Display the first few rows of the processed dataframe
akas_df.head()


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [82]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics_df['tconst'].isin(akas_df['titleId'])
keepers1



NameError: name 'keepers1' is not defined

In [ ]:
basics_df.replace({'\\N': np.nan}, inplace=True)

# Display the first few rows of the processed dataframe
basics_df.head()


In [ ]:
basics_df = basics_df[keepers]
basics_df



In [ ]:
basics_df = basics_df[basics_df['runtimeMinutes'].notna()]

basics_df = basics_df[basics_df['genres'].notna()]

basics_df = basics_df[basics_df.titleType == 'movie']

basics_df = basics_df[basics_df['startYear'].notna()]

basics_df = basics_df[basics_df['startYear'].astype(float)]
basics_df.dtypes

In [ ]:
basics_df

In [ ]:
# Filtering the basics dataframe using startYear column to keep movies between 2000 and 2021 inclusive
basics_df = basics_df[(basics_df['startYear'] >= 2000) & (basics_df['startYear'] <= 2021)]


In [ ]:
# Exclude movies that are included in the documentary category.
is_documentary = df['genres'].str.contains('documentary',case=False)
df = df[~is_documentary]


In [ ]:
ratings_df.replace({'\\N': np.nan}, inplace=True)

In [ ]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =ratings_df['tconst'].isin(akas_df['titleId'])
keepers2


In [ ]:
# Save Dataframe
akas_df.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)

# Open saved file
akas_df = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)
akas_df.head()

In [ ]:
basics_df.to_csv("Data/title.basics.csv.gz", compression='gzip' , index=False)

#open saved file
basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics_df.head()


In [ ]:
ratings_df.to_csv("Data/title.basics.csv.gz", compression='gzip' , index=False)

#open saved file
ratings_df = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
ratings_df.head()